In [1]:
%watermark -mduv

last updated: 2018-06-25 

CPython 3.6.5
IPython 6.4.0

compiler   : GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 15.6.0
machine    : x86_64
processor  : i386
CPU cores  : 2
interpreter: 64bit


# Variable scope

Usage of <code>global</code> and <code>nonlocal</code> (see <a href="https://www.python.org/dev/peps/pep-3104/">PEP 3104</a>)

The <b><code>global</code></b> statement is a declaration which holds for the <b>entire current code block</b>. It means that the listed identifiers are to be interpreted as globals. It would be impossible to assign to a global variable without <code>global</code>, although free variables may refer to globals without being declared global.

The <b><code>nonlocal</code></b> statement causes the listed identifiers to refer to <b>previously bound variables in the nearest enclosing scope excluding globals</b>. This is important because the default behavior for binding is to search the local namespace first. The statement allows encapsulated code to rebind variables outside of the local scope besides the global (module) scope.

<h1>Closures (Statically Nested Scopes)</h1>

Closures were specified in <a href="https://www.python.org/dev/peps/pep-0227/">PEP 227</a> for Python 2.2. For further informations concerning the execution model and varaible scope, refer to <a href="https://docs.python.org/3/reference/executionmodel.html">Python Language Reference</a>. 

## Examples

No <code>global</code>/<code>nonlocal</code> declaration

In [3]:
# example from http://stackoverflow.com/questions/1261875/

x = 0
def outer():
    x = 1
    def inner():
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 1
global: 0


<code>nonlocal</code> declaration in inner closure

In [4]:
x = 0
def outer():
    x = 1
    def inner():
        nonlocal x
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 2
global: 0


globals are excluded so using nonlocal with variable binding outside the function scope leads to a `SyntaxError`

In [5]:
x = 0
def outer():
    nonlocal x
    x = 1
    def inner():
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

SyntaxError: no binding for nonlocal 'x' found (<ipython-input-5-e6aa27e8879a>, line 3)

<code>global</code> declaration in inner closure

In [6]:
x = 0
def outer():
    global x
    x = 1
    def inner():
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 1
global: 1


In [7]:
x = 0
def outer():
    global x
    x = 1
    def inner():
        global x
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 2
global: 2


In [8]:
glob = 1

def foo():
    loc = 5
    print('loc in foo():', 'loc' in locals())
    print('glob in foo():', 'glob' in globals())


foo()
print('loc in global:', 'loc' in globals())    
print('glob in global:', 'foo' in globals())

loc in foo(): True
glob in foo(): True
loc in global: False
glob in global: True


In [9]:
a = 'global'

def outer():

    def len(in_var):
        print('called my len() function: ', end="")
        l = 0
        for i in in_var:
            l += 1
        return l

    a = 'local'

    def inner():
        global len
        nonlocal a
        a += ' variable'
    inner()
    print('a is', a)
    print(len(a))
    print(locals() == globals())

outer()

print(len(a))
print('a is', a)
print(locals() == globals())

a is local variable
called my len() function: 14
False
6
a is global
True


In [10]:
globals()['__builtins__']

<module 'builtins' (built-in)>

In [11]:
locals() == globals()

True

an interesting one:

In [12]:
def g():
    print(i)
i = 42
g()

42


In [13]:
%%dis
def g():
    print(i)
i = 42
g()

  1           0 LOAD_CONST               0 (<code object g at 0x3193936f0, file "<dis>", line 1>)
              2 LOAD_CONST               1 ('g')
              4 MAKE_FUNCTION            0
              6 STORE_NAME               0 (g)

  3           8 LOAD_CONST               2 (42)
             10 STORE_NAME               1 (i)

  4          12 LOAD_NAME                0 (g)
             14 CALL_FUNCTION            0
             16 POP_TOP
             18 LOAD_CONST               3 (None)
             20 RETURN_VALUE


In [17]:
def make_adder(base):
    def adder(x):
        return base + x
    return adder
add5 = make_adder(5)
add5(6)

11

In [20]:
if 'x' in globals():
    del x
    print("Deleting x from globals()")
    
def make_adder1():
    x = 2
    values = [x]
    def adder():
        x = x + 1
        values.append(x)
        print(values)
        return x
    return adder

add5 = make_adder1()
add5()

UnboundLocalError: local variable 'x' referenced before assignment

In [40]:
make_adder1.__code__.co_varnames

('x', 'adder')

In [41]:
make_adder2.__code__.co_varnames

('adder',)

In [33]:
%dis make_adder1.__code__.co_code

  1           0 LOAD_NAME                0 (make_adder1)
              2 LOAD_ATTR                1 (__code__)
              4 LOAD_ATTR                2 (co_code)
              6 RETURN_VALUE


In [39]:
if 'x' in globals():
    del x
    print("Deleting x from globals()")
    
def make_adder2():
    x = 2
    values = [x]
    def adder():
        nonlocal x
        x = x + 1
        values.append(x)
        print(values)
        return x
    return adder

add5 = make_adder2()
add5()

[2, 3]


3

In [25]:
add5()

[2, 3, 4]


4

In [26]:
dis(make_adder2)

  1           0 LOAD_NAME                0 (make_adder2)
              2 RETURN_VALUE


## Recursive closure call

In [4]:
def make_fact():
    def fact(n):
        if n == 1:
            return 1
        else:
            return n * fact(n - 1)
    return fact

In [5]:
fact = make_fact()
fact(7)

5040

In [27]:
def make_wrapper(obj):
    class Wrapper:
        def __getattr__(self, attr):
            if attr[0] != '_':
                 return getattr(obj, attr)
            else:
                raise AttributeError(attr)
    return Wrapper()

In [40]:
class Test:
     public = 2
     _private = 3

w = make_wrapper(Test())
w.public

2

In [41]:
w._private

AttributeError: _private

In [30]:
i = 6
def f(x):
    def g():
        print i
        # ...
        # skip to the next page
        # ...
        for i in x:  # ah, i *is* local to f, so this is what g sees
            pass
    g()

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-30-2e5743477a39>, line 4)